# 🛠️ Advanced Tool Use with GitHub Models (.NET)

## 📋 Learning Objectives

This notebook demonstrates enterprise-grade tool integration patterns using the Microsoft Agent Framework in .NET with GitHub Models. You'll learn to build sophisticated agents with multiple specialized tools, leveraging C#'s strong typing and .NET's enterprise features.

**Advanced Tool Capabilities You'll Master:**
- 🔧 **Multi-Tool Architecture**: Building agents with multiple specialized capabilities
- 🎯 **Type-Safe Tool Execution**: Leveraging C#'s compile-time validation
- 📊 **Enterprise Tool Patterns**: Production-ready tool design and error handling
- 🔗 **Tool Composition**: Combining tools for complex business workflows

## 🎯 .NET Tool Architecture Benefits

### Enterprise Tool Features
- **Compile-Time Validation**: Strong typing ensures tool parameter correctness
- **Dependency Injection**: IoC container integration for tool management
- **Async/Await Patterns**: Non-blocking tool execution with proper resource management
- **Structured Logging**: Built-in logging integration for tool execution monitoring

### Production-Ready Patterns
- **Exception Handling**: Comprehensive error management with typed exceptions
- **Resource Management**: Proper disposal patterns and memory management
- **Performance Monitoring**: Built-in metrics and performance counters
- **Configuration Management**: Type-safe configuration with validation

## 🔧 Technical Architecture

### Core .NET Tool Components
- **Microsoft.Extensions.AI**: Unified tool abstraction layer
- **Microsoft.Agents.AI**: Enterprise-grade tool orchestration
- **GitHub Models Integration**: High-performance API client with connection pooling

### Tool Execution Pipeline
```csharp
User Request → Agent Analysis → Tool Selection → Type Validation
                 ↓               ↓              ↓
         Parameter Binding → Tool Execution → Result Processing → Response
```

## 🛠️ Tool Categories & Patterns

### 1. **Data Processing Tools**
- **Input Validation**: Strong typing with data annotations
- **Transform Operations**: Type-safe data conversion and formatting
- **Business Logic**: Domain-specific calculation and analysis tools
- **Output Formatting**: Structured response generation

### 2. **Integration Tools** 
- **API Connectors**: RESTful service integration with HttpClient
- **Database Tools**: Entity Framework integration for data access
- **File Operations**: Secure file system operations with validation
- **External Services**: Third-party service integration patterns

### 3. **Utility Tools**
- **Text Processing**: String manipulation and formatting utilities
- **Date/Time Operations**: Culture-aware date/time calculations
- **Mathematical Tools**: Precision calculations and statistical operations
- **Validation Tools**: Business rule validation and data verification

## ⚙️ Prerequisites & Setup

**Development Environment:**
- .NET 9.0 SDK or higher
- Visual Studio 2022 or VS Code with C# extension
- GitHub Models API access

**Required NuGet Packages:**
```xml
<PackageReference Include="Microsoft.Extensions.AI" Version="9.9.0" />
<PackageReference Include="Microsoft.Extensions.AI.OpenAI" Version="9.9.0-preview.1.25458.4" />
<PackageReference Include="DotNetEnv" Version="3.1.1" />
```

**Environment Configuration (.env file):**
```env
GITHUB_TOKEN=your_github_personal_access_token
GITHUB_ENDPOINT=https://models.inference.ai.azure.com
GITHUB_MODEL_ID=gpt-4o-mini
```

Ready to build enterprise-grade agents with powerful, type-safe tool capabilities in .NET? Let's architect some professional-grade solutions! 🏢⚡

In [1]:
#r "nuget: Microsoft.Extensions.Logging.Abstractions, 10.0.0"
#r "nuget: Microsoft.Extensions.DependencyInjection.Abstractions, 10.0.0"
#r "nuget: Microsoft.Extensions.Options, 10.0.0"
#r "nuget: OpenAI, 2.8.0"

Installed Packages Microsoft.Extensions.DependencyInjection.Abstractions, 10.0.0 Microsoft.Extensions.Logging.Abstractions, 10.0.0 Microsoft.Extensions.Options, 10.0.0 OpenAI, 2.8.0

In [2]:
#r "nuget: System.ClientModel, 1.8.1"

Installed Packages system.clientmodel, 1.8.1

In [3]:
#r "nuget: Microsoft.Extensions.AI, 10.2.0"
#r "nuget: Microsoft.Extensions.AI.OpenAI, 10.2.0-preview.1.26063.2"

Installed Packages Microsoft.Extensions.AI, 10.2.0 Microsoft.Extensions.AI.OpenAI, 10.2.0-preview.1.26063.2

In [4]:
#r "/Users/lokinfey/Desktop/AOAI/Foundry/agent-framework/dotnet/src/Microsoft.Agents.AI.OpenAI/bin/Debug/net10.0/Microsoft.Agents.AI.OpenAI.dll"

In [5]:
#r "/Users/lokinfey/Desktop/AOAI/Foundry/agent-framework/dotnet/src/Microsoft.Agents.AI/bin/Debug/net10.0/Microsoft.Agents.AI.dll"

In [6]:
// #r "nuget: Microsoft.Extensions.AI.OpenAI, 9.9.0-preview.1.25458.4"

In [7]:
#r "nuget: DotNetEnv, 3.1.1"

Installed Packages DotNetEnv, 3.1.1

In [8]:
using System;
using System.ComponentModel;
using System.ClientModel;

using Microsoft.Extensions.AI;
using Microsoft.Agents.AI;
using OpenAI;

In [9]:
 using DotNetEnv;

In [10]:
Env.Load("../../../.env");

In [11]:
[Description("Provides a random vacation destination.")]
static string GetRandomDestination()
{
    var destinations = new List<string>
    {
        "Paris, France",
        "Tokyo, Japan",
        "New York City, USA",
        "Sydney, Australia",
        "Rome, Italy",
        "Barcelona, Spain",
        "Cape Town, South Africa",
        "Rio de Janeiro, Brazil",
        "Bangkok, Thailand",
        "Vancouver, Canada"
    };

    var random = new Random();
    int index = random.Next(destinations.Count);
    return destinations[index];
}

## 🎲 Define Random Destination Tool

Create a tool function that provides random vacation destinations.
The `[Description]` attribute helps the AI understand the tool's purpose and when to invoke it.

In [12]:
var github_endpoint = Environment.GetEnvironmentVariable("GITHUB_ENDPOINT") ?? throw new InvalidOperationException("GITHUB_ENDPOINT is not set.");
var github_model_id = Environment.GetEnvironmentVariable("GITHUB_MODEL_ID") ?? "gpt-4o-mini";
var github_token = Environment.GetEnvironmentVariable("GITHUB_TOKEN") ?? throw new InvalidOperationException("GITHUB_TOKEN is not set.");

## 🔑 Load Environment Configuration

Extract GitHub Models credentials and settings from environment variables.

In [13]:
var openAIOptions = new OpenAIClientOptions()
{
    Endpoint= new Uri(github_endpoint)
};

## ⚙️ Configure OpenAI Client for GitHub Models

Set up the OpenAI client options to redirect API calls to GitHub Models endpoint.

In [14]:

var openAIClient = new OpenAIClient(new ApiKeyCredential(github_token), openAIOptions);

## 🔌 Initialize OpenAI Client

Create the authenticated OpenAI client instance for GitHub Models communication.

In [15]:
AIAgent agent = new OpenAIClient(new ApiKeyCredential(github_token), openAIOptions).GetChatClient(github_model_id).AsIChatClient().AsAIAgent(
    instructions:"You are a helpful AI Agent that can help plan vacations for customers at random destinations", tools: [AIFunctionFactory.Create((Func<string>)GetRandomDestination)]);

## 🤖 Create AI Agent with Tool Integration

Build the agent pipeline and register the random destination tool using `AIFunctionFactory`.
The agent will automatically call this tool when users request travel suggestions.

In [16]:
AgentThread thread = await agent.GetNewThreadAsync();

## 🧵 Create Conversation Thread

Initialize a new thread to track tool execution and maintain conversation context.

In [17]:
Console.WriteLine(await agent.RunAsync("Plan me a day trip",thread));

I have chosen Bangkok, Thailand as your random destination for the day trip. Here's a suggested itinerary:

Morning:
- Start your day with a visit to the Grand Palace, one of Bangkok's most famous landmarks.
- Walk to Wat Phra Kaew (Temple of the Emerald Buddha) located within the Grand Palace complex.

Afternoon:
- Head to Wat Arun (Temple of Dawn) and enjoy the stunning riverside views.
- Have lunch at a local Thai restaurant and try some authentic Pad Thai or Tom Yum soup.

Evening:
- Explore the vibrant street markets like Chatuchak Weekend Market or Asiatique the Riverfront.
- End your day with a dinner cruise on the Chao Phraya River to enjoy the city's nightscape.

Would you like me to help you with accommodations, transportation, or dining reservations for this trip?


## 🚀 Execute First Request with Tool Invocation

Run the agent with a travel planning request. The agent will automatically invoke the destination tool and generate a detailed itinerary.

In [18]:
Console.WriteLine(await agent.RunAsync("I don't like that destination. Plan me another vacation.",thread));

Let's plan a day trip to Tokyo, Japan!

Morning:
- Start your day at the historic Sensō-ji Temple in Asakusa, Tokyo's oldest temple.
- Explore Nakamise Shopping Street leading to the temple for souvenirs and snacks.

Afternoon:
- Visit the Tokyo Skytree for panoramic views of the city.
- Enjoy lunch at a nearby sushi restaurant or try some local street food.

Evening:
- Head to Shibuya Crossing, one of the busiest pedestrian crossings in the world.
- Explore the vibrant Shibuya area with its shops, cafes, and entertainment options.
- Have dinner in one of the many restaurants offering Japanese cuisine.

Would you like suggestions for accommodations or transportation options in Tokyo?


## 🔄 Follow-up Request: Tool Re-invocation

Demonstrate how the agent handles follow-up requests by calling the tool again to provide an alternative destination.